In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm

import sys
sys.path.append(os.path.abspath('..'))

from tools.network import Network
from tools.decode_raw import decode_img_seg

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [ ]:
csv_path = os.path.join('..', 'data', 'train.csv')
train_path = os.path.join('..', 'data', 'train_images')
tfr_path = os.path.join('..', 'tmp', 'TFRecords', 'train')
ckpt_dir = os.path.join('..', 'tmp', 'ckpt', 'cv')

feature_dict = {
    'img': 'bytes', 
    'label': 'bytes',
    'height': 'int', 
    'width': 'int',
    'channels': 'int',
    'n_class': 'int'
}

rt_params = {
    'feature_dict': feature_dict, 
    'decode_raw': decode_img_seg,
    'tfr_path': os.path.join(tfr_path, '*.tfrecord'),
    'shuffle_buffer': 100,
    'compression': 'GZIP',
    'num_parallel_reads':3
}

readtrain_params = {
    'rt_params': rt_params, 
    'train_path': train_path,
    'epoch': 2,
    'batch_size': 32,
    'reshape': [32, 200],
    'reshape_method': 3}

arch_params = {
    'num_layers': 3,
    'feature_growth_rate': 16,
    'n_class': 5,
    'channels': 3,
    'padding': 'SAME',
    'dropout_rate':0.25
}

model_params = {
    'model_name': 'unet',
    'model_params': arch_params,
    'loss': 'cross_entropy',
    'metric': 'neg_dice',
    'optimizer': 'momentun',
    'rate': 1e-6}

network = Network()

In [ ]:
result = network.cv(
    readtrain_params, 
    model_params,
    ckpt_dir=ckpt_dir,
    nfolds=5,
    train_percentage=0.8,
    early_stopping=1,
    verbose=2)